<a href="https://colab.research.google.com/github/Zedbog/First_repository/blob/master/zadania_mysql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Inizcalizacja środowiska Google Colab
!apt-get install mysql-server > /dev/null
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'"
!pip -q install PyMySQL

 * Starting MySQL database server mysqld
   ...done.
ERROR 1045 (28000): Access denied for user 'root'@'localhost' (using password: NO)


In [2]:
# zaimporotowanie bibliotek sqlalchemy 
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

In [3]:
engine = create_engine('mysql+pymysql://root:root@/')

In [4]:
engine.execute('CREATE DATABASE cinematic')
engine.execute('USE cinematic')

ProgrammingError: ignored

In [66]:
eng = create_engine("mysql+pymysql://root:root@/cinematic")
base = declarative_base()

class Directors(base):
  __tablename__ = 'directors'

  director_id = Column(Integer, primary_key=True, autoincrement=True)
  name = Column(String(30), nullable=False)
  surname = Column(String(30), nullable=False)
  rating = Column(Integer, nullable=False)

  def __repr__(self):
    return f"<Director: id: {self.director_id}, Name: {self.name}, Surname: {self.surname}, Movie_rating: {self.rating}>"

class Movies(base):
  __tablename__ = 'movies'

  movie_id = Column(Integer, primary_key=True, autoincrement=True)
  title = Column(String(60), nullable=False)
  year = Column(Integer, nullable=False)
  category = Column(String(30), nullable=False)
  director_id = Column(Integer, nullable=False)
  rating = Column(Integer, nullable=False)

  def __repr__(self):
    return f"<Movie: id: {self.movie_id}, Title: {self.title}, Year: {self.year}, Category: {self.category}, Director_id: {self.director_id}, Director_=rating: {self.rating}>"
base.metadata.create_all(eng)

In [67]:
#Utworzenie sesji do wykonywania zapytań
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=eng)
session = Session()

In [91]:
directors = [{'name': 'Frank', 'surname': 'Darabont', 'rating': 7}, {'name':
'Francis Ford', 'surname': 'Coppola', 'rating': 8}, {'name': 'Quentin',
'surname': 'Tarantino', 'rating': 10}, {'name': 'Christopher', 'surname':
'Nolan', 'rating': 9}, {'name': 'David', 'surname': 'Fincher', 'rating': 7}]
movies = [{'title': 'The Shawshank Redemption', 'year': 1994,
'category': 'Drama', 'director_id': 1, 'rating': 8}, {'title': 'The Green Mile',
'year': 1999, 'category': 'Drama', 'director_id': 1, 'rating': 6}, {'title': 'The Godfather', 'year': 1972, 'category': 'Crime', 'director_id': 2, 'rating': 7},
{'title': 'The Godfather III', 'year': 1990, 'category': 'Crime', 'director_id':
2, 'rating': 6}, {'title': 'Pulp Fiction', 'year': 1994, 'category': 'Crime',
'director_id': 3, 'rating': 9}, {'title': 'Inglourious Basterds', 'year': 2009,
'category': 'War', 'director_id': 3, 'rating': 8}, {'title': 'The Dark Knight',
'year': 2008, 'category': 'Action', 'director_id': 4, 'rating': 9}, {'title':
'Interstellar', 'year': 2014, 'category': 'Sci-fi', 'director_id': 4, 'rating': 8},
{'title': 'The Prestige', 'year': 2006, 'category': 'Drama', 'director_id': 4,
'rating': 10}, {'title': 'Fight Club', 'year': 1999, 'category': 'Drama',
'director_id': 5, 'rating': 7}, {'title': 'Zodiac', 'year': 2007, 'category':
'Crime', 'director_id': 5, 'rating': 5}, {'title': 'Seven', 'year': 1995,
'category': 'Drama', 'director_id': 5, 'rating': 8}, {'title': 'Alien 3', 'year':
1992, 'category': 'Horror', 'director_id': 5, 'rating': 5}]

In [92]:
def insert_data(session, base, params):
  session.add(base(**params))
  session.commit()

In [93]:
for director in directors:
  insert_data(session, Directors, director)
for movie in movies:
  insert_data(session, Movies, movie)

In [ ]:
for director in session.query(Directors).all():
  print(director) 

In [ ]:
#4. Wypisanie wszystkich filmów z kategorii Drama
for movie in session.query(Movies).filter(Movies.category == 'Drama'):
  print(movie) 

In [ ]:
#5. Wypisanie tytułów filmów z kategorii Crime, wyprodukownaych po 1994
for movie in session.query(Movies).filter(Movies.category == 'Crime').filter(Movies.year >= 1994):
  print(movie.title)

In [ ]:
""""6. Wypisanie kategorii wszystkich filmów oraz ich ranking dla filmów, 
które zostały wyprodukowane w latach 2000 - 2010 oraz których ranking jest większy niż 7.
sortując po rankingu"""

for movie in session.query(Movies).filter(Movies.year >= 2000).filter(Movies.year <= 2010).filter(Movies.rating >= 7).order_by(Movies.rating.desc()):
  print(f"Movie category: {movie.category}, Rating: {movie.rating}")

In [ ]:
""" Wypisz nazwiska wszystkich reżyserów, których ranking jest większy bądź
równy 6, a ich imię zaczyna się na literę 'D' lub kończy się na literę 'n'."""
for director in session.query(Directors).filter(Directors.rating >= 6):
  if (director.name[0] == 'D' or director.name[-1] == 'n'):
    print(director.surname)

In [15]:
""" Badanie tabeli za pomocą pandas profiling"""
from pandas_profiling import ProfileReport
import pandas as pd
from pathlib import Path
from ipywidgets import widgets
from google.colab import files

In [73]:
from sqlalchemy import join
from sqlalchemy.sql import select

j = join(Movies, Directors, Movies.director_id == Directors.director_id)

In [88]:
conn = eng.connect()
s = select([Movies])
# print(conn.execute(s).fetchall())
for movie in session.query(Movies).all():
  # print(movie)
  session.delete(movie)
for director in session.query(Directors).all():
  session.delete(director)

In [ ]:
for movie in session.query(Movies).all():
  print(movie)
for director in session.query(Directors).all():
    print(director)

In [ ]:
conn = eng.connect()
sql_db = pd.read_sql_query(select([Movies, Directors]).select_from(j),conn)

In [ ]:
df = pd.DataFrame(sql_db, columns=['movie_id', 'title', 'year', 'category', 'rating', 'director_id', 'name', 'surname'])

In [ ]:
import sys
!pip install -U pip
!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

In [61]:
profile = ProfileReport(sql_db, 
                        title = 'Movies',
                        html={'style': {'full_width':True}},
                        sort='None')

In [ ]:
display(sql_db)

In [ ]:
profile.to_file('movies.html')

# files.download('movies.html')